In [34]:
# Importing required libraries
import os
import logging
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting
import plotly.graph_objects as go
import re

In [35]:
from google.colab import drive

# Mount Google Drive (for Colab)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
# Update these paths to match your Google Drive directories
INPUT_FOLDER = '/content/drive/MyDrive/weekend_project/Interaction_Folder/B_DNA '    # Input folder containing interaction CSV files
OUTPUT_FOLDER = '/content/drive/MyDrive/weekend_project/graph/B-DNA'    # Output folder to save graphs and plots

# Create Output Folder if it doesn't exist
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
    logging.info(f"Created output folder at {OUTPUT_FOLDER}.")
else:
    logging.info(f"Output folder already exists at {OUTPUT_FOLDER}.")


In [37]:
# Function to extract numeric residue number from identifier
def get_numeric_residue_number(residue_identifier):
    if isinstance(residue_identifier, int):
        return residue_identifier
    elif isinstance(residue_identifier, str):
        match = re.search(r'\d+', residue_identifier)
        if match:
            return int(match.group())
        else:
            return None
    else:
        return None

In [38]:
# Function to process interaction file and construct graph
def process_interaction_file(file_path):
    logging.info(f"Processing interaction file: {file_path}")

    try:
        interaction_df = pd.read_csv(file_path)
        logging.info(f"Loaded interaction data with {len(interaction_df)} interactions.")
    except Exception as e:
        logging.error(f"Error loading interaction data: {e}")
        return None, None

    G = nx.Graph()
    for idx, row in interaction_df.iterrows():
        residue_i = str(row['Residue_Number_1'])
        residue_j = str(row['Residue_Number_2'])
        interaction_strength = row['Interaction_Strength']
        G.add_edge(residue_i, residue_j, weight=interaction_strength)

    logging.info(f"Constructed DSG with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

    residue_info = {}
    for node in G.nodes():
        residue_number = get_numeric_residue_number(node)
        if residue_number is not None:
            residue_info[node] = {'Residue Sequence Number': residue_number}
        else:
            residue_info[node] = {'Residue Sequence Number': None}

    return G, residue_info


In [39]:
# Function to analyze graph and return analysis DataFrame
def analyze_graph(G):
    Imin_values = np.arange(0, 6.5, 0.5)
    k_values = range(1, 10)
    data_records = []

    for Imin in Imin_values:
        G_filtered = nx.Graph()
        for u, v, d in G.edges(data=True):
            if d['weight'] > Imin:
                G_filtered.add_edge(u, v, weight=d['weight'])
        degrees = dict(G_filtered.degree())

        for k in k_values:
            hubs = [node for node, degree in degrees.items() if degree > k]
            H = len(hubs)
            data_records.append({'Imin': Imin, 'k': k, 'H': H})

    df_analysis = pd.DataFrame(data_records)
    return df_analysis


In [40]:
# Function to generate static 3D line plot for a specific k value
def generate_3d_plot(df_analysis, output_folder, fixed_k):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    sequence_names = df_analysis['sequence'].unique()
    colors = plt.cm.get_cmap('tab10', len(sequence_names))

    for i, sequence in enumerate(sequence_names):
        df_sequence = df_analysis[df_analysis['sequence'] == sequence]
        df_fixed_k = df_sequence[df_sequence['k'] == fixed_k]
        ax.plot(df_fixed_k['Imin'], df_fixed_k['H'], df_fixed_k['k'],
                label=f'{sequence} (k={fixed_k})', color=colors(i))

    ax.set_xlabel('Interaction Strength (Imin)')
    ax.set_ylabel('Number of Hubs (H)')
    ax.set_zlabel('k')
    ax.set_title(f'3D Line Plot for k = {fixed_k}')
    ax.legend()
    plt.tight_layout()

    plot_path = os.path.join(output_folder, f'3D_line_plot_k_{fixed_k}.png')
    plt.savefig(plot_path)
    plt.close()
    logging.info(f"Saved 3D line plot for k = {fixed_k} to {plot_path}")



In [41]:
# Function to generate interactive 3D line plot for a specific k value
def generate_3d_plot_interactive(df_analysis, output_folder, fixed_k):
    traces = []
    sequence_names = df_analysis['sequence'].unique()
    colors = px.colors.qualitative.Plotly

    for i, sequence in enumerate(sequence_names):
        df_sequence = df_analysis[df_analysis['sequence'] == sequence]
        df_filtered = df_sequence[df_sequence['k'] == fixed_k]
        trace = go.Scatter3d(
            x=df_filtered['Imin'],
            y=[fixed_k] * len(df_filtered),
            z=df_filtered['H'],
            mode='lines',
            name=f'{sequence} (k={fixed_k})',
            line=dict(color=colors[i % len(colors)], width=3)
        )
        traces.append(trace)

    layout = go.Layout(
        title=f'Interactive 3D Line Plot for k = {fixed_k}',
        scene=dict(
            xaxis=dict(title='Interaction Strength (Imin)'),
            yaxis=dict(title='Hub Degree Cutoff (k)'),
            zaxis=dict(title='Number of Hubs (H)'),
        )
    )

    fig = go.Figure(data=traces, layout=layout)
    plot_path = os.path.join(output_folder, f'3D_plot_line_interactive_k_{fixed_k}.html')
    fig.write_html(plot_path)
    logging.info(f"Saved interactive 3D line plot for k = {fixed_k} to {plot_path}")


In [42]:
# Function to generate combined 2D plot for k values [2, 4, 5, 6]
def generate_2d_plot(df_analysis, output_folder, k_values=[2, 4, 5, 6]):
    sequence_names = df_analysis['sequence'].unique()
    colors = plt.cm.get_cmap('tab10', len(sequence_names))

    for fixed_k in k_values:
        plt.figure(figsize=(8, 6))
        for i, sequence in enumerate(sequence_names):
            df_sequence = df_analysis[(df_analysis['sequence'] == sequence) & (df_analysis['k'] == fixed_k)]
            sns.lineplot(x='Imin', y='H', data=df_sequence, marker='o', label=sequence, color=colors(i))

        plt.xlabel('Interaction Strength (Imin)')
        plt.ylabel('Number of Hubs (H)')
        plt.title(f'H vs Imin for k = {fixed_k} (All Files)')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plot_path = os.path.join(output_folder, f'H_vs_Imin_k_{fixed_k}_combined.png')
        plt.savefig(plot_path)
        plt.close()
        logging.info(f"Saved combined 2D plot for k={fixed_k} to {plot_path}")


In [43]:
# Function to visualize DNA structure with spring layout for each file
def visualize_dna_structure_with_spring_layout(interaction_df, output_folder, sequence_name):
    G = nx.Graph()
    for _, row in interaction_df.iterrows():
        G.add_edge(row['Residue_Number_1'], row['Residue_Number_2'], weight=row['Interaction_Strength'])

    pos = nx.spring_layout(G, weight='weight')

    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]

    edge_trace = go.Scatter(x=edge_x, y=edge_y, mode='lines', line=dict(width=0.5, color='#888'), hoverinfo='none')
    node_trace = go.Scatter(x=node_x, y=node_y, mode='markers',
                            marker=dict(showscale=True, colorscale='YlGnBu', size=10, color=list(dict(G.degree).values()),
                                        colorbar=dict(thickness=15, title='Node Connections', xanchor='left', titleside='right')))

    fig = go.Figure(data=[edge_trace, node_trace], layout=go.Layout(title=f'Interactive DNA Structure for {sequence_name}',
                                                                    showlegend=False, hovermode='closest',
                                                                    margin=dict(b=20, l=5, r=5, t=40),
                                                                    xaxis=dict(showgrid=False, zeroline=False),
                                                                    yaxis=dict(showgrid=False, zeroline=False)))

    plot_path = os.path.join(output_folder, f'dna_structure_spring_layout_{sequence_name}.html')
    fig.write_html(plot_path)
    logging.info(f"Saved DNA structure plot for {sequence_name} to {plot_path}")


In [45]:
# Main function to process all files together and generate combined plots
def main():
    try:
        interaction_files = [file for file in os.listdir(INPUT_FOLDER) if file.endswith('.csv')]
    except Exception as e:
        logging.error(f"Error accessing input folder {INPUT_FOLDER}: {e}")
        return

    if not interaction_files:
        logging.warning(f"No interaction CSV files found in the input folder: {INPUT_FOLDER}")
        return

    logging.info(f"Found {len(interaction_files)} interaction file(s) in the input folder.")

    combined_analysis = pd.DataFrame()
    all_interaction_dfs = []

    for interaction_file in interaction_files:
        input_file_path = os.path.join(INPUT_FOLDER, interaction_file)
        sequence_name = os.path.splitext(interaction_file)[0].replace('_interactions', '')

        G, residue_info = process_interaction_file(input_file_path)
        if G is None or residue_info is None:
            continue

        df_analysis = analyze_graph(G)
        df_analysis['sequence'] = sequence_name
        combined_analysis = pd.concat([combined_analysis, df_analysis], ignore_index=True)

        interaction_df = pd.read_csv(input_file_path)
        interaction_df['sequence'] = sequence_name
        all_interaction_dfs.append((interaction_df, sequence_name))

    # Generate 3D plots for fixed k values
    for fixed_k in [3, 4, 5, 6]:
        generate_3d_plot(combined_analysis, OUTPUT_FOLDER, fixed_k)
        generate_3d_plot_interactive(combined_analysis, OUTPUT_FOLDER, fixed_k)

    # Generate 2D plots for k values [2, 4, 5, 6]
    generate_2d_plot(combined_analysis, OUTPUT_FOLDER)

    # Visualize DNA structure for each input file separately
    for interaction_df, sequence_name in all_interaction_dfs:
        visualize_dna_structure_with_spring_layout(interaction_df, OUTPUT_FOLDER, sequence_name)

    logging.info("All files have been processed and combined plots generated.")

if __name__ == "__main__":
    main()

<ipython-input-40-7f7f23b1f610>:7: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

<ipython-input-42-204a2b549f17>:4: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

